In [1]:
import pandas as pd
import os
from scipy.stats import kruskal
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
maple_train_df = pd.read_csv('maple_train.csv')
maple_test_df = pd.read_csv('maple_test.csv')
maple_eval_df = pd.read_csv('maple_eval.csv')


In [3]:

path = './PathwayFeatures'
pathway_names = []
for i, attrlist in enumerate(os.listdir(path)):
    attributes = pd.read_csv(path+'/'+ attrlist)
    attributes = attributes["Attrib"].tolist()
    pathway_name = attrlist.split('-')[0]
    pathway_names.append(pathway_name)

    maple_train_df.loc[maple_train_df['Path'] == pathway_name.replace('_', ' '), 'Class'] = 'positive'
    maple_test_df.loc[maple_test_df['Path'] == pathway_name.replace('_', ' '), 'Class'] = 'positive'
    maple_eval_df.loc[maple_eval_df['Path'] == pathway_name.replace('_', ' '), 'Class'] = 'positive'

    maple_train_df.loc[maple_train_df['Path'] != pathway_name.replace('_', ' '), 'Class'] = 'negative'
    maple_test_df.loc[maple_test_df['Path'] != pathway_name.replace('_', ' '), 'Class'] = 'negative'
    maple_eval_df.loc[maple_eval_df['Path'] != pathway_name.replace('_', ' '), 'Class'] = 'negative'

#     train_df_prep = maple_train_df[attributes]
#     test_df_prep = maple_test_df[attributes]
#     eval_df_prep = maple_eval_df[attributes]

#     train_df_prep.to_csv('Datasets/'+pathway_name+'_train.csv', index=False)
#     test_df_prep.to_csv('Datasets/'+pathway_name+'_test.csv', index=False)
#     eval_df_prep.to_csv('Datasets/'+pathway_name+'_eval.csv', index=False)

In [4]:
# https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
# by: @invoktheshell
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)):
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, recall_score, precision_score

def calc_metrics(y_true, y_pred, model_name, pw_name, dataset_name, fold_num):
    acc = accuracy_score(y_true, y_pred)
    tp, fp, tn, fn = perf_measure(y_true, y_pred)
    recall = precision_score(y_true, y_pred)
    specifity = tn / (tn + fp)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    # print(precision, specifity, tp,fp,tn,fn)

    try:
        fpr = fp / (fp + tn)
    except:
        fpr = 0

    try:
        tpr = tp / (tp + fn)
    except:
        tpr = 0

    try:
        fnr = fn / (fn + tp)
    except:
        fnr = 0

    try:
        fdr = fp / (fp + tp)
    except:
        fdr = 0

    return model_name, pw_name, dataset_name, fold_num, acc, f1, mcc, tp, fp, tn, fn, recall, precision, specifity, fpr, tpr, fnr, fdr

In [5]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [6]:
#.3 Metabolic pathway modeling and analysis
# The 10-fold cross-validation and bagging were used on all training process,
# and the metrics accuracy, percentage of correctly classified instances (CCI),
# true positive (TP) rate, false positive (FP) rate, false negative (FN) rate, recall,
# specificity, F-measure, false discovery rate (FDR), and Matthews coefficient correlation (MCC)
# were used to evaluate the predictive performances.
from tqdm import tqdm
res = []

for pathway_name in tqdm(pathway_names):
    train_df = pd.read_csv('Datasets/'+ pathway_name+'_train.csv')
    # test_df = pd.read_csv('Datasets/'+ pathway_name+'_test.csv')
    # eval_df = pd.read_csv('Datasets/'+ pathway_name+'_eval.csv')

    print('pathway: ', pathway_name)
    # print(len(train_df[train_df["Class"]=='positive']))

    train_df["Class"] = train_df["Class"].apply(lambda x: 1 if x=='positive' else 0)
    # test_df["Class"] = test_df["Class"].apply(lambda x: 1 if x=='positive' else 0)
    # eval_df["Class"] = eval_df["Class"].apply(lambda x: 1 if x=='positive' else 0)

    names = ['svc', 'nb', 'dt', 'mlp', 'knn', 'rf', 'sgd']
    models = [BaggingClassifier(SVC()),
              BaggingClassifier(GaussianNB(), n_jobs=-1),
              BaggingClassifier(DecisionTreeClassifier()),
#               BaggingClassifier(MLPClassifier()),
              BaggingClassifier(KNeighborsClassifier(n_jobs=-1),n_jobs=-1),
              BaggingClassifier(RandomForestClassifier(n_jobs=-1),n_jobs=-1),
              BaggingClassifier(SGDClassifier(n_jobs=-1),n_jobs=-1),]

    # 'shuffle' is added, bc 'random_state' is set
    skf = StratifiedKFold(n_splits=10, random_state=1337, shuffle=True)
    X = train_df.drop('Class', axis=1)
    y = train_df["Class"]

    # res = []
    for i in tqdm(range(len(models))):
        fold_num=0
        for train_index, test_index in skf.split(X, y):
            fold_num+=1
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            y_train, y_test = y.loc[train_index], y.loc[test_index]
            models[i].fit(X_train, y_train)
            preds = models[i].predict(X_test)
            # print(y_test[y_test[]])
            # res.append(calc_metrics(y_test.tolist(), preds.tolist(), names[i], pathway_name, 'test_df', fold_num))
            res.append(calc_metrics(y_test.tolist(), preds.tolist(), names[i], pathway_name, 'train_df', fold_num))

res_df = pd.DataFrame(res, columns=["model_name", "pw_name", "dataset_name", "fold_num", "acc", "f1", "mcc", "tp", "fp", "tn", "fn", "recall", "precision", "specifity", "fpr", "tpr", "fnr", "fdr"])
# res_dict[pathway_name] = res_df

file_name = '10fold_results.csv'
res_df.to_csv(file_name, index=False)
print(file_name, "is done.")

del y_test, y_train, X_test, X_train, X, y

  0%|          | 0/6 [00:00<?, ?it/s]

pathway:  energy



  0%|          | 0/6 [00:00<?, ?it/s]

pathway:  protein_modification



  0%|          | 0/6 [00:00<?, ?it/s]

...

pathway:  phospholipid



  0%|          | 0/6 [00:00<?, ?it/s]

pathway:  nucleotide_sugar



 67%|██████▋   | 4/6 [00:40<00:20, 10.15s/it]/home/jack/anaconda3/envs/Epias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jack/anaconda3/envs/Epias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/jack/anaconda3/envs/Epias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jack/anaconda3/envs/Epias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning:

10fold_results.csv is done.


In [29]:
res_df[res_df["model_name"] == "mlp"]

,model_name,pw_name,dataset_name,fold_num,acc,f1,mcc,tp,fp,tn,fn,recall,precision,specifity,fpr,tpr,fnr,fdr,score


In [30]:
# res_df.loc[res_df["model_name"] == "rf", "model_name"] = "sgd"
# res_df.loc[res_df["model_name"] == "knn", "model_name"] = "rf"
# res_df.loc[res_df["model_name"] == "mlp", "model_name"] = "knn"
res_df.to_csv('10fold_results.csv', index=False)

In [ ]:
# The best classifiers for each metabolic pathway were ranked using the Kruskal-Wallis test.
# Bu test algoritmaları nasıl sıralıyor anlamadım, benim anladığım fark var mı diye bakıyor dağılımlarda


res_df = pd.read_csv('10fold_results.csv')
res_rows = res_df.drop('fold_num', axis=1).groupby('model_name').mean()
for i in range(len(res_rows.index)):
    for j in range(i+1, len(res_rows.index)):
        # print(pw_name, "-", str(res_rows.index[i]), "-", str(res_rows.index[j]))
        stat, p = kruskal(res_rows.loc[str(res_rows.index[i])], res_rows.loc[str(res_rows.index[j])])
        print('Statistics=%.3f, p=%.3f' % (stat, p))
        if p < 0.05:
            print("statistically significant p value is found: ", str(res_rows.index[i], str(res_rows.index[j])))
        print("")

In [45]:
res_df.groupby(['pw_name','model_name'], as_index=False).mean().sort_values(["pw_name", "score"], ascending=False)

,pw_name,model_name,fold_num,acc,f1,mcc,tp,fp,tn,fn,recall,precision,specifity,fpr,tpr,fnr,fdr,score
67,secondary_metabolite,knn,5.5,0.986701,0.395663,0.489199,2.6,0.0,539.0,7.3,1.000000,1.000000,1.000000,0.000000,0.263333,0.736667,0.000000,1.382364
71,secondary_metabolite,svc,5.5,0.983969,0.192424,0.289982,1.1,0.0,539.0,8.8,0.800000,0.800000,1.000000,0.000000,0.113333,0.886667,0.000000,1.176393
69,secondary_metabolite,rf,5.5,0.982329,0.038182,0.064451,0.2,0.0,539.0,9.7,0.200000,0.200000,1.000000,0.000000,0.021111,0.978889,0.000000,1.020511
70,secondary_metabolite,sgd,5.5,0.983422,0.260430,0.318543,1.7,0.9,538.1,8.2,0.656667,0.656667,0.998330,0.001670,0.174444,0.825556,0.343333,0.343852
66,secondary_metabolite,dt,5.5,0.983058,0.241282,0.274227,1.6,1.0,538.0,8.3,0.485000,0.485000,0.998145,0.001855,0.163333,0.836667,0.315000,0.224340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,amine_and_polyamine,svc,5.5,0.994899,0.916997,0.918930,16.0,0.0,530.1,2.8,1.000000,1.000000,1.000000,0.000000,0.851170,0.148830,0.000000,1.911896
3,amine_and_polyamine,rf,5.5,0.992530,0.875477,0.879670,14.7,0.0,530.1,4.1,1.000000,1.000000,1.000000,0.000000,0.781287,0.218713,0.000000,1.868007
4,amine_and_polyamine,sgd,5.5,0.994352,0.910586,0.911025,16.1,0.4,529.7,2.7,0.977341,0.977341,0.999245,0.000755,0.856433,0.143567,0.022659,1.504939
0,amine_and_polyamine,dt,5.5,0.992713,0.881525,0.883023,15.3,0.5,529.6,3.5,0.969722,0.969722,0.999057,0.000943,0.813158,0.186842,0.030278,1.374238


In [46]:
# The best classifiers were pre-selected, seeking a low FP & high F-measure and CCI

best_models = {}    # {"pathway": [best 3 models]}
models = []
scores = {}
res_df["score"] =  res_df["acc"] + res_df["f1"] - res_df["fp"]
best_df = res_df.groupby(['pw_name','model_name'], as_index=False).mean().sort_values(["pw_name", "score"], ascending=False)
for pw in pathway_names:
    best_models[pw] = best_df[best_df["pw_name"] == pw].iloc[:3]["model_name"].tolist()
print(best_models)

{'energy': ['sgd', 'svc', 'rf'], 'protein_modification': ['rf', 'svc', 'dt'], 'amino_acid': ['rf', 'svc', 'knn'], 'nitrogen': ['svc', 'sgd', 'rf'], 'aromatic_compound': ['sgd', 'knn', 'svc'], 'secondary_metabolite': ['knn', 'svc', 'rf'], 'carbohydrate': ['rf', 'svc', 'knn'], 'lipid': ['svc', 'rf', 'knn'], 'carotenoid': ['knn', 'svc', 'sgd'], 'amine_and_polyamine': ['knn', 'svc', 'rf'], 'phospholipid': ['knn', 'svc', 'rf'], 'nucleotide_sugar': ['knn', 'svc', 'sgd']}


In [47]:
# estimators for bagging classifier
bagging_cls = {'svc': ('svc', BaggingClassifier(SVC())),
               'dt': ('dt', BaggingClassifier(DecisionTreeClassifier())),
#                'mlp': ('mlp', BaggingClassifier(MLPClassifier())),
               'nb' : ('nb', BaggingClassifier(GaussianNB())),
               'rf': ('rf', BaggingClassifier(RandomForestClassifier())),
               'knn': ('knn', BaggingClassifier(KNeighborsClassifier())),
               'sgd': ('sgd', BaggingClassifier(SGDClassifier()))}

In [58]:
params_rf = {'rf__base_estimator__bootstrap': [True],
             'rf__base_estimator__max_depth': [80, 100],
             'rf__base_estimator__max_features': [2, 3],
             'rf__base_estimator__min_samples_leaf': [3, 4],
             'rf__base_estimator__min_samples_split': [8, 10],
             'rf__base_estimator__n_estimators': [100, 200],
             'rf__base_estimator__n_jobs': [-1]
            }

params_SVC={'svc__base_estimator__C': [1, 100, 1000],
            'svc__base_estimator__gamma': [1, 0.01, 0.0001],
            'svc__base_estimator__kernel': ['rbf']
           }


params_MLP = {'mlp__base_estimator__hidden_layer_sizes': [(10,30,10),(20,)],
              'mlp__base_estimator__activation': ['tanh', 'relu'],
              'mlp__base_estimator__solver': ['sgd', 'adam'],
              'mlp__base_estimator__alpha': [0.0001, 0.05],
              'mlp__base_estimator__learning_rate': ['constant','adaptive']
             }

params_DT = { 
             'dt__base_estimator__max_depth': list(range(6,9)),
             'dt__base_estimator__min_samples_leaf': list(range(4,10,2))
            }

params_KNN = { 
             'knn__base_estimator__n_neighbors': list(range(3, 7)),
             'knn__base_estimator__n_jobs': [-1]
             }

params_SGD = {
    'sgd__base_estimator__penalty': ['l2'],
    'sgd__base_estimator__n_jobs': [-1],
#     'sgd__base_estimator__C': [1e-7, 1e-5, 1e-3],
}

all_params = {"rf": params_rf, 
              "svc": params_SVC, 
#               "mlp": params_MLP, 
              "dt": params_DT, 
              "knn": params_KNN, 
              "sgd": params_SGD}


In [59]:
voting_models_w_params = {}    # ["pathway": (voting model, parameters)]

for pw_name, models in best_models.items():

    estimators = []
    parameters = {}

    for model in models:
        # model's estimator
        estimators.append(bagging_cls[model])

        # there is no param for NB
        if model in all_params.keys():
            for param, values in all_params[model].items():
                parameters[param] = values

    # pramameters =

    voting_model = VotingClassifier(estimators=estimators, voting='soft')
    voting_models_w_params[pw_name] = (voting_model, parameters)

In [56]:
voting_models_w_params

{'energy': (VotingClassifier(estimators=[('sgd',
                                BaggingClassifier(base_estimator=SGDClassifier())),
                               ('svc', BaggingClassifier(base_estimator=SVC())),
                               ('rf',
                                BaggingClassifier(base_estimator=RandomForestClassifier()))],
                   voting='soft'),
  {'sgd__base_estimator__penalty': ['l2'],
   'sgd__base_estimator__n_jobs': [-1],
   'sgd__base_estimator__C': [1e-07, 1e-05, 0.001],
   'svc__base_estimator__C': [1, 100, 1000],
   'svc__base_estimator__gamma': [1, 0.01, 0.0001],
   'svc__base_estimator__kernel': ['rbf'],
   'rf__base_estimator__bootstrap': [True],
   'rf__base_estimator__max_depth': [80, 100],
   'rf__base_estimator__max_features': [2, 3],
   'rf__base_estimator__min_samples_leaf': [3, 4],
   'rf__base_estimator__min_samples_split': [8, 10],
   'rf__base_estimator__n_estimators': [100, 200],
   'rf__base_estimator__n_jobs': [-1]}),
 'protein_

In [ ]:
import json

# parameters tuning of combined algorithms

best_scores = {}    # "model": best scores
best_params = {}    # "model": best parameters
params_rows = []
for pathway_name in pathway_names:
    df = pd.read_csv('Datasets/' + pathway_name+'_train.csv')

    voting_model = voting_models_w_params[pathway_name][0]
    parameters = voting_models_w_params[pathway_name][1]

    print("Starting for", pathway_name)
    clf = GridSearchCV(voting_model, parameters, n_jobs=-1, verbose=3, scoring='f1_macro', cv=3)
    clf.fit(df.drop('Class', axis=1), df["Class"])

    best_scores[pathway_name] = clf.best_score_
    best_params[pathway_name] = clf.best_params_
    print(clf.best_score_)
    print(clf.best_params_)

with open('best_params.json', 'w') as fp:
    json.dump(best_params, fp)

In [21]:
for (model, params) in voting_models_w_params.values():
    print(model)

In [22]:
voting_models_w_tuned_params = voting_models_w_params.copy()

for pw, (model, params) in voting_models_w_params.items():

    voting_models_w_params[pw][0] = model.set_params(**best_params[pw])
    voting_models_w_params[pw][1] = best_params[pw]

best_models

In [22]:
for pw, (model, params) in voting_models_w_params.items():
    